In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from IPython.display import clear_output
import numpy as np
import pickle

In [10]:
headings = pd.read_pickle('/Users/chinmayshrivastava/Desktop/OSCA/V4.0/scraping/headings.pickle')
kwid = pd.read_pickle('/Users/chinmayshrivastava/Desktop/OSCA/V4.0/analysisofkeyelements/kwid.pickle')
kwdata = pd.read_pickle('/Users/chinmayshrivastava/Desktop/OSCA/V4.0/analysisofkeyelements/synonymsdata.pickle')

In [3]:
def updatedict(somedict):
    from nltk import everygrams
    tempd = {}
    somedictlist = sorted(somedict, key=len)
    for element in somedictlist:
        temp = everygrams(element.split(' '), max_len=len(element.split(' '))-1)
        temp = sorted(temp, key=len)
        temp = [' '.join(element) for element in temp]
        for ele in temp:
            if ele not in tempd:
                tempd[ele] = somedict[element]
            else:
                continue
    for element in tempd:
        if element in somedict:
            somedict[element] -= tempd[element]
    return somedict

In [4]:
headings.head()

,tag,headingtext,subheading,nextheading,paragraphs
0,topic,Is Blockchain-Based Watch-to-Earn Monetization...,2,None,[]
1,h3,@ishantechIshanOnTech,None,None,[0]
2,h2,What Is Happening?,None,3,[1]
3,h2,What is Coub.com?,None,4,"[2, 3]"
4,h2,How is Coubâs Web3 Video Space Going to Work?,None,5,"[4, 5, 6, 7]"


In [7]:
tempkwid = {}
getback = {}
i=0
for kw in kwid:
    tempkwid[kw] = i
    getback[i] = kw
    i+=1
hcontent = []
length = len(headings.index)
for hid in headings.index:
    clear_output(wait=True)
    print(hid/length)
    vector = CountVectorizer(vocabulary=tempkwid, ngram_range=(1, 4))
    h = vector.fit_transform([headings.iloc[hid]['headingtext']])
    hcount = h.toarray()
    temp = {}
    for elementid in range(len(hcount[0])):
        if hcount[0, elementid]!=0:
            temp[kwdata[kwid[getback[elementid]]]['name']] = hcount[0, elementid]
    temp = updatedict(temp)
    hcontent.append(temp)

0.9999845399873228


In [11]:
headings['headingelementcontent'] = hcontent

In [21]:
headings.head(10)

,tag,headingtext,subheading,nextheading,paragraphs,headingelementcontent
0,topic,Is Blockchain-Based Watch-to-Earn Monetization...,2,None,[],{'blockchain': 1}
1,h3,@ishantechIshanOnTech,None,None,[0],{}
2,h2,What Is Happening?,None,3,[1],{}
3,h2,What is Coub.com?,None,4,"[2, 3]",{}
4,h2,How is Coubâs Web3 Video Space Going to Work?,None,5,"[4, 5, 6, 7]",{'web3': 1}
5,h2,How is this Web3 Video Platform going to help ...,None,6,"[8, 9]",{'web3': 1}
6,h2,Final Thoughts on Web3 Video platforms?,None,None,"[10, 11, 12]",{'web3': 1}
7,topic,Pros and Cons of Development On Polygon,9,None,[],{'polygon': 1}
8,h3,@serkhitrovSergei Khitrov,None,None,"[13, 14]",{}
9,h2,How Polygon solves Ethereum's scalability problem,None,10,"[15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]","{'ethereum': 1, 'polygon': 1, 'scalability': 1}"


In [14]:
paragraphs = pd.read_pickle('paragraphswithanalysis.pickle')

In [19]:
def call_related_headings(headingid, headings, listofrelatedids=None, nextheading=False):
    if listofrelatedids is None:
        listofrelatedids = []
    if not nextheading:
        if headings.iloc[headingid]['subheading']!=None:
            subheadingid = headings.iloc[headingid]['subheading']
            listofrelatedids.append(subheadingid)
            listofrelatedids = call_related_headings(subheadingid, headings, listofrelatedids, True)
    if nextheading:
        if headings.iloc[headingid]['nextheading']!=None:
            nextheadingid = headings.iloc[headingid]['nextheading']
            listofrelatedids.append(nextheadingid)
            listofrelatedids = call_related_headings(nextheadingid, headings, listofrelatedids, True)
    return listofrelatedids

In [29]:
kwsinpgs = []
kwsinhs = []
wordcount = []
length = len(headings.index)
for hid in headings.index:
    wordcount_ = 0
    clear_output(wait=True)
    print(hid/length)
    subheadings = call_related_headings(hid, headings)
    pgids = headings.iloc[hid]['paragraphs']
    for shid in subheadings:
        pgids.extend(headings.iloc[shid]['paragraphs'])
    tempkwsinpgs = {}
    tempkwsinhs = {}
    for pgid in pgids:
        wordcount_ += paragraphs.iloc[pgid]['count']
        temp = paragraphs.iloc[pgid]['keyelements']
        for element in temp:
            if element in tempkwsinpgs:
                tempkwsinpgs[element] += temp[element]
            else:
                tempkwsinpgs[element] = temp[element]
    for shid in subheadings:
        wordcount_ += len(headings.iloc[shid]['headingtext'].split())
        temp = headings.iloc[shid]['headingelementcontent']
        for element in temp:
            if element in tempkwsinhs:
                tempkwsinhs[element] += temp[element]
            else:
                tempkwsinhs[element] = temp[element]
    kwsinpgs.append(tempkwsinpgs)
    kwsinhs.append(tempkwsinhs)
    wordcount.append(wordcount_)

0.9999845399873228


In [31]:
headings['kwsinsubhs'] = kwsinhs
headings['kwsinpgs'] = kwsinpgs
headings['count'] = wordcount

In [32]:
headings.head()

,tag,headingtext,subheading,nextheading,paragraphs,headingelementcontent,kwsinsubhs,kwsinpgs,count
0,topic,Is Blockchain-Based Watch-to-Earn Monetization...,2,None,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, ...",{'blockchain': 1},{'web3': 3},"{'blockchain': 6, 'nfts': 12, 'web3': 21, 'nft...",1930.0
1,h3,@ishantechIshanOnTech,None,None,[0],{},{},{'web3': 1},11.0
2,h2,What Is Happening?,None,3,[1],{},{},"{'blockchain': 1, 'nfts': 1, 'web3': 1}",89.0
3,h2,What is Coub.com?,None,4,"[2, 3]",{},{},{'web3': 1},120.0
4,h2,How is Coubâs Web3 Video Space Going to Work?,None,5,"[4, 5, 6, 7]",{'web3': 1},{},"{'nft marketplace': 1, 'nfts': 2, 'wallet': 1,...",258.0


In [26]:
with open('headingswithanalysis.pickle', 'wb') as file:
    pickle.dump(headings, file)

In [27]:
paragraphs.head()

,tag,text,keyelements,count
0,p,"Covering the latest events, insights and views...",{'web3': 1},11.0
1,p,"Coub.com, a micro-video streaming social netwo...","{'blockchain': 1, 'nfts': 1, 'web3': 1}",89.0
2,p,Coub is a video-sharing website that is access...,{},57.0
3,p,"On Coub, users can take any video, trim the be...",{'web3': 1},63.0
4,p,In order to preserve the intrinsic fluidity of...,"{'nft marketplace': 1, 'nfts': 1, 'wallet': 1,...",76.0
